# Collect metadata from CrossRef

Intended to collect DOI metadata in batches from CrossRef via habanero, but not working atm

issue reported

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import datetime, time, sys
import json
import urllib.parse
import pandas as pd
import configparser
from pprint import pprint
from itertools import zip_longest
from tqdm import tqdm, tqdm_notebook

from ATB.ATB.DBConnection import DBConnection
from habanero import Crossref

tqdm.pandas(tqdm_notebook)

In [3]:
db_path = 'data/main.db'
db_table = 'pkp_crossref'
db_col_names = ['doi', 'timestamp', 'cr_object']
db_col_types = ['TEXT', 'TEXT', 'TEXT']
unique_col = "doi"

db_con = DBConnection(db_path=db_path,
                      db_table=db_table,
                      db_col_names=db_col_names,
                      db_col_types=db_col_types,
                      unique_col=unique_col)

In [4]:
cr = Crossref()

In [5]:
full_df = pd.read_csv("PKP_temp.csv", encoding = 'utf8')
dois = full_df[full_df.doi_resolve_status == "200"].doi

In [6]:
def parse_response(doi, now, cr_response):

    row = {'doi':str(doi),
           'timestamp':str(now),
           'cr_object':json.dumps(cr_response),
          }
    
    return row

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [72]:
try:
    res = cr.works(ids=a[0][0:4], cursor="*")
except:
    print('error')
    res = None

In [50]:
doi_groups = list(grouper(dois, 100))
for i, i_dois in tqdm(enumerate(doi_groups), total=len(doi_groups)):
    now = datetime.datetime.now()
    
    try:
        cr_response = cr.works(ids=i_dois)
    except:
        pass
    
    for response in cr_response:
        # Create DB entry
        row = parse_response(response['message']['DOI'], now, response['message'])
        db_con.save_row(row)
    
    # Do some nice API things
    if False:
        new = datetime.datetime.now()
        delta = new - now
        m, s = divmod(i_max-i, 60)
        h, m = divmod(m, 60)

        if delta.seconds < 1:
            time.sleep(1- delta.total_seconds())


  0%|          | 0/1216 [00:00<?, ?it/s]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/asura/.virtualenvs/altmetrics/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




HTTPError: 404 Client Error: Not Found for url: https://api.crossref.org/works/10.15150/lt.2003.1517